In [ ]:
!pip install python-dotenv

import os
import csv

from bs4 import BeautifulSoup
from dotenv import load_dotenv
from typing import List, Union
from requests import get, Response

load_dotenv()

PATH = 'pars_autoru.csv'
URL = 'https://auto.ru/moskva/cars/all/'
COOKIE = os.getenv('COOKIE')

HEADERS = {
    'user-agent': os.getenv('USER_AGENT'),
    'accept': os.getenv('ACCEPT'),
    'Accept-Language': 'ru',
    'accept-encoding': 'accept - encoding: gzip, deflate, br',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="101", "Opera";v="87"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'Upgrade-Insecure-Requests': '1',
    'Cookie': "autoru_gdpr=1; suid=c884a3c0767127496a9ebc040a3a4ce9.6209fa50b6d2a9f56642f5b08f671160; "
    "_csrf_token=305b66f4692deac7ed0d8eb3a0c26df1a090df3ec74c96e4; "
    "autoru_sid=a%3Ag67c743692nqnqv4215hm4sr696vs7tu.6aa14550ccf50cd8ffe2c9b8163928a6%7C1741112169778.604800.6r1e81CbwDLv69kIupjuzQ.7A7p2P3brGQVeTblXl1x45djo-ugX05hjMzAHXLRLOI; "
    "autoruuid=g67c743692nqnqv4215hm4sr696vs7tu.6aa14550ccf50cd8ffe2c9b8163928a6; "
    "from=direct; yandex_login=; "
    "i=pzRIaqugGPXgRZNJ3jTpx0/XDTMmcXLQ9rqlgyVr07BEfWc8jzmzrSeXolAfpiiL5CalUuqhBz7MiIuoqNYFNl6Mpvo=; "
    "yandexuid=8285180061601070617; yaPassportTryAutologin=1; "
    "fp=918bb73e5ab734d215b9a218709c44ea%7C1741112175597; los=1; bltsr=1; coockoos=4; "
    "autoru_crashreport={%22route_name%22:%22listing%22%2C%22app_id%22:%22af-desktop-search%22}; "
    "crookie=lQAiCFaRQWBQurFwn6+5Jplf49oSvljI347V717Lizen8S4eZDKwi0e7TOpeHr9U92lDlTBTEvA3kNOO5yrn4AJgufQ=; "
    "cmtchd=MTc0MTExMjE4NTUyOQ==; popups-popup-car-history-for-free-shown-count=1; "
    "_ym_uid=1741112385974008306; _ym_isad=1; autoru_sso_blocked=1; "
    "_yasc=xD63l+RCawFl+2QRavF0Px6uh7ojGMKTtpJHILvy9B9akIgwIS68TsKOCtBgWNrHpw==; "
    "Session_id=noauth:1741155427; "
    "sessar=1.1199.CiDzZFUz1wzj8l991kPjWjFazSTD71XT77JDgrBKmhedsw.NeKhNzfFC_9RwhMQNryyOWBVB54hCf_K7sr1uoWwJrI; "
    "ys=c_chck.329368023; mda2_beacon=1741155427892; sso_status=sso.passport.yandex.ru:synchronized; "
    "autoru_sso_redirect_blocked=1; _ym_d=1741155434; count-visits=1; "
    "from_lifetime=1741155438392; "
    'layout-config={"screen_height":900,"screen_width":1440,"win_width":885,"win_height":812}'
}

def get_pages_amount(content: bytes) -> int:
    soup = BeautifulSoup(content, 'html.parser')

    pagination_block = soup.find(
        'span',
        class_='ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination__pages'
    )
    if not pagination_block:
        print("Внимание: блок пагинации не найден. Будем считать, что страница одна.")
        return 1

    pages = pagination_block.contents
    return len(pages)


def get_content(html: bytes) -> List[dict]:
    soup = BeautifulSoup(html, 'html.parser')

    items = soup.find_all('div', class_='ListingItem')
    if not items:
        items = soup.find_all('div', class_='ListingItem__description')

    cars = []
    for item in items:
        summary_block = item.find('div', class_='ListingItem__summary')
        link_block = item.find('a', class_='ListingItemTitle__link')
        price_block = item.find('div', class_='ListingItemPrice__content')
        year_block = item.find('div', class_='ListingItem__yearBlock')
        summary_text = summary_block.get_text(strip=True) if summary_block else 'N/A'
        link = link_block.get('href') if link_block else 'N/A'

        if price_block:
            price_text = price_block.get_text(strip=True).replace('\xa0', '').replace('₽', '')
        else:
            price_text = 'N/A'

        year_text = year_block.get_text(strip=True) if year_block else 'N/A'

        car = {
            'car': summary_text,
            'url': link,
            'price': price_text,
            'year': year_text,
        }
        cars.append(car)
    return cars


def get_html(url: str, headers: dict, params: Union[None, dict] = None) -> Response:

    try:
        return get(url, headers=headers, params=params)
    except Exception as error:
        raise ConnectionError(f'При выполнении запроса произошла ошибка: {error}')


def parse(url: str) -> List[dict]:
    url = url or URL
    html = get_html(url, HEADERS)

    if html.status_code != 200:
        print(f"Сайт вернул статус-код {html.status_code}")
        return []

    pages_amount = get_pages_amount(html.content)

    cars = []
    for i in range(1, pages_amount + 1):
        print(f'Парсим {i} страницу из {pages_amount}...')
        html = get_html(url, HEADERS, params={'page': i})
        if html.status_code != 200:
            print(f"Страница {i} вернула статус-код {html.status_code}, пропускаем.")
            continue

        page_cars = get_content(html.content)
        cars.extend(page_cars)

    print(f'Получены данные по {len(cars)} авто.')

    def parse_year(car):
        try:
            return int(car['year'])
        except:
            return 0
    sorted_cars = sorted(cars, key=parse_year, reverse=True)
    return sorted_cars


def save_to_file(data) -> None:

    with open(PATH, 'w', newline='', encoding='utf-8') as file:
        w = csv.writer(file, delimiter=';')
        w.writerow(['Car', 'Link', 'Price (RUR)', 'Year'])
        for car in data:
            w.writerow([
                car['car'],
                car['url'],
                car['price'],
                car['year']
            ])


def main():
    url_input = input('URL: ').strip()
    data = parse(url_input)
    if data:
        save_to_file(data)


if __name__ == '__main__':
    main()


URL: 
Парсим 1 страницу из 11...
Парсим 2 страницу из 11...
Парсим 3 страницу из 11...
Парсим 4 страницу из 11...
Парсим 5 страницу из 11...
Парсим 6 страницу из 11...
Парсим 7 страницу из 11...
Парсим 8 страницу из 11...
Парсим 9 страницу из 11...
Парсим 10 страницу из 11...
Парсим 11 страницу из 11...
Получены данные по 407 авто.
